# Relevant

## 1. `base.py::BasicObject.INIT`

## 2. `utils/pyramid.py`

## 3. `utils/vocab.py`


# Less Relevant 

## 4. `base.py::BasicObject.OBJECT_TO_PICKLE`

## 5. `base.py::BasicObject.INIT_FROM_PICKLE`

## 6. `folder.py`, `text.py`, `sentence.py`, `token.py` (partial)

## 7. `utils/infrastructure.py::Part CODE & Input and Output `

In [ ]:
from nlptext.base import BasicObject

# ########### Wiki All ###########
CORPUSPath = 'corpus/WikiTotal/'
GroupIden = '.txt'
TextType   = 'line'
Text2SentMethod  = 're'
Sent2TokenMethod = 'sep- '
TOKENLevel = 'word'
anno = False
annoKW = {}

BasicObject.INIT(CORPUSPath, GroupIden, TextType,
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel,
                 anno, annoKW)

```
'File'
corpus/WikiTotal/WikiTotal7k_v2.txt
100000 -- 3 190 2019-06-27 14:40:58.170793
200000 -- 1 94 2019-06-27 14:41:21.156057
300000 -- 1 139 2019-06-27 14:41:42.477667
400000 -- 2 299 2019-06-27 14:42:03.825076
500000 -- 2 184 2019-06-27 14:42:24.798024
```

In [2]:
from nlptext.base import BasicObject

# ########### Wiki All ###########
CORPUSPath = 'corpus/WikiEng/'
GroupIden = '.txt'
TextType   = 'line'
Text2SentMethod  = 're'
Sent2TokenMethod = 'sep- '
TOKENLevel = 'word'
anno = False
annoKW = {}

BasicObject.INIT(CORPUSPath, GroupIden, TextType,
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel,
                 anno, annoKW)

'File'
corpus/WikiEng/wiki_en.txt
100000 -- 1 4442 2019-06-27 16:14:21.655432
200000 -- 1 18068 2019-06-27 16:20:37.503412
300000 -- 1 453 2019-06-27 16:25:31.245183
400000 -- 1 842 2019-06-27 16:29:37.455739
500000 -- 1 2541 2019-06-27 16:33:20.352198
600000 -- 1 5285 2019-06-27 16:36:45.087139
700000 -- 1 6559 2019-06-27 16:39:49.359356
800000 -- 1 4660 2019-06-27 16:42:47.864291
900000 -- 1 4273 2019-06-27 16:45:37.061889
1000000 -- 1 893 2019-06-27 16:48:20.266300
1100000 -- 1 548 2019-06-27 16:51:00.526036
1200000 -- 1 5721 2019-06-27 16:53:26.978164
1300000 -- 1 913 2019-06-27 16:55:54.312335
1400000 -- 1 870 2019-06-27 16:58:44.156900
1500000 -- 1 971 2019-06-27 17:02:19.521715
1600000 -- 1 499 2019-06-27 17:05:35.338467
1700000 -- 1 564 2019-06-27 17:08:38.328902
1800000 -- 1 751 2019-06-27 17:11:39.533079
1900000 -- 1 1057 2019-06-27 17:14:38.481220
2000000 -- 1 2679 2019-06-27 17:17:34.452599
2100000 -- 1 514 2019-06-27 17:20:41.928869
2200000 -- 1 666 2019-06-27 17:23:40.534

In [3]:
BasicObject.SENT

{'Sent2TokenMethod': 'sep- ',
 'EndIDXTokens': array([       12,        22,        53, ..., 257789044, 257789070,
        257789077], dtype=uint32),
 'EndIDXTokens_File': array([        51,         93,        266, ..., 1433929701, 1433929815,
        1433929853], dtype=uint32),
 'length': 11199643}

# 1. A Brief Overview for Pyramid


Show the pyramid structures: Corpus, Folder, Text, Sentence, Token

In [8]:
import numpy as np

corpus = [['1.病史：患者为63岁女性，慢性病程，急性加重。',
'既往有“高脂血症”病史。',
'2.因“反复脐周疼痛2年余，再发并加重1周”入院。'],
['3.体查：血压128/63mmHg，神志清楚，浅表淋巴结无肿大，口唇无苍白。',
'双侧扁桃体无肿大、充血。咽无充血。颈静脉无怒张。'],
['双肺呼吸音清晰，未闻及干湿性啰音，无胸膜摩擦音。',
'心率62次/分，律齐，各瓣膜区未闻及病理性杂音。',
'腹部平坦，未见胃肠型及蠕动波，腹壁柔软，脐周压痛，无反跳痛，未扪及包块，肝脾肋下未扪及，Murphy征（-）',
'肝肾区无叩击痛，移动性浊音（-），肠鸣音4次/分。',
'双下肢无浮肿。四肢肌力、肌张力正常，生理反射存在，病理反射未引出。']]
# print(len(corpus))
corpus


[['1.病史：患者为63岁女性，慢性病程，急性加重。', '既往有“高脂血症”病史。', '2.因“反复脐周疼痛2年余，再发并加重1周”入院。'],
 ['3.体查：血压128/63mmHg，神志清楚，浅表淋巴结无肿大，口唇无苍白。', '双侧扁桃体无肿大、充血。咽无充血。颈静脉无怒张。'],
 ['双肺呼吸音清晰，未闻及干湿性啰音，无胸膜摩擦音。',
  '心率62次/分，律齐，各瓣膜区未闻及病理性杂音。',
  '腹部平坦，未见胃肠型及蠕动波，腹壁柔软，脐周压痛，无反跳痛，未扪及包块，肝脾肋下未扪及，Murphy征（-）',
  '肝肾区无叩击痛，移动性浊音（-），肠鸣音4次/分。',
  '双下肢无浮肿。四肢肌力、肌张力正常，生理反射存在，病理反射未引出。']]

In [14]:
def utf8len(s):
    return len(s.encode('utf-8'))

In [16]:
import os

SENT = []

processed_data_path = 'processed_data.txt'


DTU = {}
LTU = []
index2freq = []
# data = np.zeros(5000, dtype= np.uint32)
token_num_in_corpus = 0

if os.path.isfile(processed_data_path):
    os.remove(processed_data_path)
        
for text in corpus:
    for strSent in text:
        strTokens = [i for i in strSent]
        for token in strTokens:
            if token not in DTU:
                # deal with new words
                token_idx  = len(DTU)
                DTU[token] = token_idx
                index2freq.append(1)
                LTU.append(token)
            else:
                # deal with old words
                token_idx = DTU[token]
                index2freq[token_idx] += 1
            # data.append(token_idx)
            # data[token_num_in_corpus] = token_idx
            token_num_in_corpus = token_num_in_corpus + 1
            
        with open(processed_data_path, 'a') as f:
            line_sentence = ' '.join(strTokens) + '\n'
            f.write(line_sentence)
            SENT.append(utf8len(line_sentence))
            
            
            
data = data[: token_num_in_corpus]
print(len(data))
# print(data)
print(DTU)
print(LTU)
print(index2freq)

283
{'1': 0, '.': 1, '病': 2, '史': 3, '：': 4, '患': 5, '者': 6, '为': 7, '6': 8, '3': 9, '岁': 10, '女': 11, '性': 12, '，': 13, '慢': 14, '程': 15, '急': 16, '加': 17, '重': 18, '。': 19, '既': 20, '往': 21, '有': 22, '“': 23, '高': 24, '脂': 25, '血': 26, '症': 27, '”': 28, '2': 29, '因': 30, '反': 31, '复': 32, '脐': 33, '周': 34, '疼': 35, '痛': 36, '年': 37, '余': 38, '再': 39, '发': 40, '并': 41, '入': 42, '院': 43, '体': 44, '查': 45, '压': 46, '8': 47, '/': 48, 'm': 49, 'H': 50, 'g': 51, '神': 52, '志': 53, '清': 54, '楚': 55, '浅': 56, '表': 57, '淋': 58, '巴': 59, '结': 60, '无': 61, '肿': 62, '大': 63, '口': 64, '唇': 65, '苍': 66, '白': 67, '双': 68, '侧': 69, '扁': 70, '桃': 71, '、': 72, '充': 73, '咽': 74, '颈': 75, '静': 76, '脉': 77, '怒': 78, '张': 79, '肺': 80, '呼': 81, '吸': 82, '音': 83, '晰': 84, '未': 85, '闻': 86, '及': 87, '干': 88, '湿': 89, '啰': 90, '胸': 91, '膜': 92, '摩': 93, '擦': 94, '心': 95, '率': 96, '次': 97, '分': 98, '律': 99, '齐': 100, '各': 101, '瓣': 102, '区': 103, '理': 104, '杂': 105, '腹': 106, '部': 107, '平': 108, '坦': 109, '见': 

In [24]:
sentIdx = np.cumsum(SENT)
sentIdx

array([  88,  136,  228,  356,  452,  548,  638,  840,  934, 1066])

In [23]:
sent_idx = 2 # the three line
sent_pos_start = sentIdx[sent_idx - 1]
print(sent_pos_start)

136


In [29]:
def get_line_with_position(processed_data_path,start_position):
    with open(processed_data_path, 'r') as f:
        f.seek(start_position)
        line = f.readline() # there is no 's' in f.readline()
        # last_pos = f.tell()
        # f.close()
    return line

In [11]:
new_index2freq = np.sort(index2freq)[::-1]
new_index2freq

array([21, 12,  9,  6,  6,  6,  5,  5,  4,  4,  4,  4,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1])

In [7]:
newidx2oldidx = np.argsort(index2freq)[::-1]
newidx2oldidx# [10]

array([ 13,  19,  61,  83,  12,  85,  87,   2,  29,  31,  36,  26,   0,
        62,   9,   1,  34,  68,  48, 104,   8, 115,  63, 112, 106, 103,
        54,  98,  44,  46,  92,  49,  28,  72,  73,  86,  79,  97,  33,
       135, 149, 136, 121, 145,  17,  18, 127, 148, 137,  23,   4, 153,
         3, 124,  14,  55,   6,   7,  53,  59,   5,  52,  57,  51,  10,
        58,  11,  60,  56,  27,  50,  47,  30,  25,  24,  32,  65,  35,
        22,  37,  38,  21,  20,  39,  40,  41,  42,  43,  16,  45,  15,
        64, 157,  66, 114, 117, 118, 119, 120, 122, 123, 125, 126, 128,
       129, 130, 131, 132, 133, 134, 138, 139, 140, 141, 142, 143, 144,
       146, 147, 150, 151, 152, 154, 155, 116, 113,  67, 111,  69,  70,
        71,  74,  75,  76,  77, 156,  80,  81,  82,  84,  88,  89,  90,
        91,  93,  94,  95,  96,  99, 100, 101, 102, 105, 107, 108, 109,
       110,  78])

In [8]:
oldidx2newidx = np.zeros(len(newidx2oldidx), dtype= int)

for new_idx, old_idx in enumerate(newidx2oldidx):
    oldidx2newidx[old_idx] = new_idx
    
oldidx2newidx

array([ 12,  15,   7,  52,  50,  60,  56,  57,  20,  14,  64,  66,   4,
         0,  54,  90,  88,  44,  45,   1,  82,  81,  78,  49,  74,  73,
        11,  69,  32,   8,  72,   9,  75,  38,  16,  77,  10,  79,  80,
        83,  84,  85,  86,  87,  28,  89,  29,  71,  18,  31,  70,  63,
        61,  58,  26,  55,  68,  62,  65,  59,  67,   2,  13,  22,  91,
        76,  93, 126,  17, 128, 129, 130,  33,  34, 131, 132, 133, 134,
       157,  36, 136, 137, 138,   3, 139,   5,  35,   6, 140, 141, 142,
       143,  30, 144, 145, 146, 147,  37,  27, 148, 149, 150, 151,  25,
        19, 152,  24, 153, 154, 155, 156, 127,  23, 125,  94,  21, 124,
        95,  96,  97,  98,  42,  99, 100,  53, 101, 102,  46, 103, 104,
       105, 106, 107, 108, 109,  39,  41,  48, 110, 111, 112, 113, 114,
       115, 116,  43, 117, 118,  47,  40, 119, 120, 121,  51, 122, 123,
       135,  92])

In [10]:
new_LTU = []
for new_idx in range(len(LTU)):
    new_LTU.append(LTU[newidx2oldidx[new_idx]])
    
print(new_LTU)
new_DTU = {}
for new_idx, token in enumerate(new_LTU):
    new_DTU[token] = new_idx
    
print(new_DTU)

['，', '。', '无', '音', '性', '未', '及', '病', '2', '反', '痛', '血', '1', '肿', '3', '.', '周', '双', '/', '理', '6', '动', '大', '肠', '腹', '区', '清', '分', '体', '压', '膜', 'm', '”', '、', '充', '闻', '张', '次', '脐', '（', '力', '-', '扪', '肢', '加', '重', '下', '肌', '）', '“', '：', '射', '史', '肝', '慢', '楚', '者', '为', '志', '巴', '患', '神', '表', 'g', '岁', '淋', '女', '结', '浅', '症', 'H', '8', '因', '脂', '高', '复', '唇', '疼', '有', '年', '余', '往', '既', '再', '发', '并', '入', '院', '急', '查', '程', '口', '出', '苍', '蠕', '壁', '柔', '软', '跳', '包', '块', '脾', '肋', 'M', 'u', 'r', 'p', 'h', 'y', '征', '肾', '叩', '击', '移', '浊', '鸣', '4', '浮', '四', '正', '常', '生', '存', '在', '波', '型', '白', '胃', '侧', '扁', '桃', '咽', '颈', '静', '脉', '引', '肺', '呼', '吸', '晰', '干', '湿', '啰', '胸', '摩', '擦', '心', '率', '律', '齐', '各', '瓣', '杂', '部', '平', '坦', '见', '怒']
{'，': 0, '。': 1, '无': 2, '音': 3, '性': 4, '未': 5, '及': 6, '病': 7, '2': 8, '反': 9, '痛': 10, '血': 11, '1': 12, '肿': 13, '3': 14, '.': 15, '周': 16, '双': 17, '/': 18, '理': 19, '6': 20, '动': 21, '大': 22, '肠': 23, '腹

In [11]:
for idx, oldidx in enumerate(data):
    data[idx] = oldidx2newidx[oldidx]
data

array([ 12,  15,   7,  52,  50,  60,  56,  57,  20,  14,  64,  66,   4,
         0,  54,   4,   7,  90,   0,  88,   4,  44,  45,   1,  82,  81,
        78,  49,  74,  73,  11,  69,  32,   7,  52,   1,   8,  15,  72,
        49,   9,  75,  38,  16,  77,  10,   8,  79,  80,   0,  83,  84,
        85,  44,  45,  12,  16,  32,  86,  87,   1,  14,  15,  28,  89,
        50,  11,  29,  12,   8,  71,  18,  20,  14,  31,  31,  70,  63,
         0,  61,  58,  26,  55,   0,  68,  62,  65,  59,  67,   2,  13,
        22,   0,  91,  76,   2,  93, 126,   1,  17, 128, 129, 130,  28,
         2,  13,  22,  33,  34,  11,   1, 131,   2,  34,  11,   1, 132,
       133, 134,   2, 157,  36,   1,  17, 136, 137, 138,   3,  26, 139,
         0,   5,  35,   6, 140, 141,   4, 142,   3,   0,   2, 143,  30,
       144, 145,   3,   1, 146, 147,  20,   8,  37,  18,  27,   0, 148,
       149,   0, 150, 151,  30,  25,   5,  35,   6,   7,  19,   4, 152,
         3,   1,  24, 153, 154, 155,   0,   5, 156, 127,  23, 12

In [42]:
TEXT_DICT = {}
TEXT_DICT['NUMSents'] = []
TEXT_DICT['EndIDXSents'] = []

SENT_DICT = {}
SENT_DICT['NUMTokens'] = []
SENT_DICT['EndIDXTokens'] = []

TOKEN_DICT = {}
TOKEN_DICT['DATAToken'] = []

for text in corpus:
    # get text feature
    
    lenText = len(text)
    TEXT_DICT['NUMSents'].append(lenText)
    try:
        TEXT_DICT['EndIDXSents'].append(SENT_DICT['EndIDXTokens'][-1] + lenText)
    except:
        TEXT_DICT['EndIDXSents'].append(lenText)
    for sent in text:
        lenSent = len(sent)
        SENT_DICT['NUMTokens'].append(lenSent)
        try:
            SENT_DICT['EndIDXTokens'].append(SENT_DICT['EndIDXTokens'][-1] + lenSent)
        except:
            SENT_DICT['EndIDXTokens'].append(lenSent)
        
        
        TOKEN_DICT['DATAToken'].extend([token for token in sent])
               

print('Text  Level Dictionary')
print(TEXT_DICT)
print()
print('Sent  Level Dictionary')
print(SENT_DICT)
print()
print('Token Level Dictionary')
print(TOKEN_DICT)

Text  Level Dictionary
{'NUMSents': [3, 2, 5], 'EndIDXSents': [3, 63, 128]}

Sent  Level Dictionary
{'NUMTokens': [24, 12, 25, 38, 24, 24, 24, 54, 25, 33], 'EndIDXTokens': [24, 36, 61, 99, 123, 147, 171, 225, 250, 283]}

Token Level Dictionary
{'DATAToken': ['1', '.', '病', '史', '：', '患', '者', '为', '6', '3', '岁', '女', '性', '，', '慢', '性', '病', '程', '，', '急', '性', '加', '重', '。', '既', '往', '有', '“', '高', '脂', '血', '症', '”', '病', '史', '。', '2', '.', '因', '“', '反', '复', '脐', '周', '疼', '痛', '2', '年', '余', '，', '再', '发', '并', '加', '重', '1', '周', '”', '入', '院', '。', '3', '.', '体', '查', '：', '血', '压', '1', '2', '8', '/', '6', '3', 'm', 'm', 'H', 'g', '，', '神', '志', '清', '楚', '，', '浅', '表', '淋', '巴', '结', '无', '肿', '大', '，', '口', '唇', '无', '苍', '白', '。', '双', '侧', '扁', '桃', '体', '无', '肿', '大', '、', '充', '血', '。', '咽', '无', '充', '血', '。', '颈', '静', '脉', '无', '怒', '张', '。', '双', '肺', '呼', '吸', '音', '清', '晰', '，', '未', '闻', '及', '干', '湿', '性', '啰', '音', '，', '无', '胸', '膜', '摩', '擦', '音', '。', '心', '

In [13]:
token_list = []
for newidx in data:
    token_list.append(new_LTU[newidx])
print(token_list)

['1', '.', '病', '史', '：', '患', '者', '为', '6', '3', '岁', '女', '性', '，', '慢', '性', '病', '程', '，', '急', '性', '加', '重', '。', '既', '往', '有', '“', '高', '脂', '血', '症', '”', '病', '史', '。', '2', '.', '因', '“', '反', '复', '脐', '周', '疼', '痛', '2', '年', '余', '，', '再', '发', '并', '加', '重', '1', '周', '”', '入', '院', '。', '3', '.', '体', '查', '：', '血', '压', '1', '2', '8', '/', '6', '3', 'm', 'm', 'H', 'g', '，', '神', '志', '清', '楚', '，', '浅', '表', '淋', '巴', '结', '无', '肿', '大', '，', '口', '唇', '无', '苍', '白', '。', '双', '侧', '扁', '桃', '体', '无', '肿', '大', '、', '充', '血', '。', '咽', '无', '充', '血', '。', '颈', '静', '脉', '无', '怒', '张', '。', '双', '肺', '呼', '吸', '音', '清', '晰', '，', '未', '闻', '及', '干', '湿', '性', '啰', '音', '，', '无', '胸', '膜', '摩', '擦', '音', '。', '心', '率', '6', '2', '次', '/', '分', '，', '律', '齐', '，', '各', '瓣', '膜', '区', '未', '闻', '及', '病', '理', '性', '杂', '音', '。', '腹', '部', '平', '坦', '，', '未', '见', '胃', '肠', '型', '及', '蠕', '动', '波', '，', '腹', '壁', '柔', '软', '，', '脐', '周', '压', '痛', '，', '无', '反', '跳', '痛',

In [43]:
sentId = 0
StartIdx = SENT_DICT['EndIDXTokens'][sentId-1] if sentId != 0 else 0 # this is more faster
EndIdx   = SENT_DICT['EndIDXTokens'][sentId]

print(StartIdx, EndIdx)
print(''.join(TOKEN_DICT['DATAToken'][StartIdx: EndIdx]))
print(corpus[0][0])

0 24
1.病史：患者为63岁女性，慢性病程，急性加重。
1.病史：患者为63岁女性，慢性病程，急性加重。


# 2. Corpus to Folders and Folder to Texts

In [ ]:
import os

def CorpusFoldersReader(CORPUSPath, iden = None):
    # file is the priority
    if iden:
        corpusFiles = [i for i in os.listdir(CORPUSPath) if iden in i]
        return {os.path.join(CORPUSPath, fd): '' for fd in corpusFiles}, 'File'
    else:
        results = [x for x in os.walk(CORPUSPath) if x[2]]
        return {i[0]: i[2] for i in results},                            'Dir'
    
CORPUSPath = 'corpus/ner/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = None)[0]
CorpusFolders

# 3. Folder to Texts

Each text is a tuple for strText, SSET, orig_file_name, anno_file_name

In [ ]:
from nlptext.utils.pyramid import CorpusFoldersReader, FolderTextsReaders

FolderTextsReaders

In [ ]:
CORPUSPath = 'corpus/ner/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = None)[0]

print(CorpusFolders)

folderPath =  list(CorpusFolders.keys())[0]
fileNames  = CorpusFolders[folderPath]

textType = 'file'

TOKENLevel = 'char'
anno = '.Entity'
annoKW = {
    'sep': '\t',
    'notZeroIndex': 0,
}


FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)

strText_SSET_O_A = list(FolderTexts)[0]
    
    
strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
print(strText)
print(SSETText)
print(origTextName)
print(annoTextName)

In [ ]:
CORPUSPath = 'corpus/medpos/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = None)[0]

# print(CorpusFolders)

folderPath =  list(CorpusFolders.keys())[0]
fileNames  = CorpusFolders[folderPath]

textType = 'file'

TOKENLevel = 'char'
anno = '.UMLSTag'
annoKW = {
    'sep': '\t',
    'notZeroIndex': 0,
}


FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)

strText_SSET_O_A = list(FolderTexts)[0]
    
    
strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
print(strText)
print(SSETText)
print(origTextName)
print(annoTextName)

In [ ]:
CORPUSPath = 'corpus/boson/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = '.txt')[0]

# print(CorpusFolders)

folderPath =  list(CorpusFolders.keys())[0]
fileNames  = CorpusFolders[folderPath]

textType = 'line'

TOKENLevel = 'char'
anno = 'embed'

FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)

strText_SSET_O_A = list(FolderTexts)[0]
     
strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
print(strText)
print(SSETText)
print(origTextName)
print(annoTextName)

The strText is then be spilted into sentences.

If SSET exists, we need to match SSET and the splited sentences.

# 4. Text to Sentences

In [ ]:
import re

##################################################################################################TEXT-SENT
def reCutText2Sent(text, useSep = False):
    
    ###################### Remove some weird chars #######################
    text = re.sub('\xa0', '', text)
    
    ############# The Issue of Spaces
    ###################### Convert the Spaces between two English Letters to 'ⴷ' #################
    # Take care of Spaces
    text = re.sub(r'(?<=[A-Za-z])\s+(?=[|A-Za-z])', 'ⴷ',  text)
    
    ###################### Convert the S+ spaces to '〰' #################
    text = re.sub(' {2}', '〰', text ).strip()
    if useSep == ' ':
        # if using space to sep the words
        text = text.replace('\t','').replace('〰', ' ')
    elif useSep == '\t':
        # if using tab to sep the words, removing all spaces
        text = text.replace(' ','').replace('〰', '')
    else:
        # if there is no sep char for Chinese, remove single space, and then convert space+ to single space
        text = text.replace('\t','').replace(' ', '',).replace('〰', ' ')
        
    # convert the spaces between English letters to single spaces
    text = text.replace('ⴷ', ' ')
    
    # Other Things
    text = re.sub('([。！!;；])([^”])', r"\1\n\2",text) 
    text = re.sub('(\.{6})([^”])',    r"\1\n\2",text) 
    text = re.sub('(\…{2})([^”])',    r"\1\n\2",text)
    
    # The \n within " " is not considered
    text = '"'.join( [ x if i % 2 == 0 else x.replace('\n', '') 
                         for i, x in enumerate(text.split('"'))] )
    text = re.sub( '\n+', '\n', text ).strip() # replace '\n+' to '\n'
    text = text.replace('\\n', '\n')
    text = text.split("\n")
    text = [sent.strip() for sent in text]
    # text = [sent.replace(' ', '').replace('\\n', '') for sent in text]
    return [sent for sent in text if len(sent)>=2]


text = '浙江在线杭州4月25日讯(记者施宇翔 通讯员 方英)毒贩很“时髦”,用微信交易毒品。没料想警方也很“潮”,将计就计,一举将其擒获。记者从杭州江干区公安分局了解到,经过一个多月的侦查工作,江干区禁毒专案组抓获吸贩毒人员5名,缴获“冰毒”400余克,毒资30000余元,扣押汽车一辆。黑龙江籍男子钱某长期落脚于宾馆、单身公寓,经常变换住址。他有一辆车,经常半夜驾车来往于杭州主城区的各大宾馆和单身公寓,并且常要活动到凌晨6、7点钟,白天则在家里呼呼大睡。钱某不寻常的特征,引起了警方注意。禁毒大队通过侦查,发现钱某实际上是在向落脚于宾馆和单身公寓的吸毒人员贩送“冰毒”。'
print(text)
reCutText2Sent(text)

In [ ]:
def segText2Sents(text, method = 'whole', **kwargs):
    
    '''
    text:
        1. textfilepath. 2. text-level string
    method: 
        1. 'whole': when text is a text-level string,then use this text-level string as sent-level string directly.
                    and return text = [sent-level string].
        2. `funct`: when method is a function, whose input is a text-level string,
                    then return text = funct(text) = [..., sent-level string, ...]
        3. 'line' : string. when text is filepath where each line is a sentence
                    then return a generator text = generate(text), item is a sent-level string.        
    '''
    if os.path.isfile(text):
        if method == 'line':
            text = lineCutText2Sent(text)
            return text
        else:
            text = fileReader(text)
    if method == 'whole':
        return [text]
    elif method == 're':
        return reCutText2Sent(text, **kwargs)
    else:
        return method(text, **kwargs)

# 5. Sentence to Tokens

In [ ]:
def segSent2Tokens(sent, method = 'iter'):
    return [i for i in sent]

# 6. Deal with Annotation

In [ ]:

strText = '''结肠多发息肉。\n患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。'''

print(strText)

strAnnoText = '''标注文本名称:/Users/zhangling/Documents/新标的数据530/529李选-已检查/Entity/patient4378.txt\n标注文本字数统计:87\n多发息肉\t3\t6\t疾病\n慢性\t16\t17\t修饰\n多发息肉\t30\t33\t疾病\n3月余\t34\t36\t修饰\n无阳性体征\t44\t48\t不确定\n'''
print(strAnnoText)

# BIOES

sep = '\t'
SSETText = [sset.split('\t') for sset in strAnnoText.split('\n') if sep in sset]

notZeroIndex = 1 

sset = SSETText[0]

strAnno = sset[0]
s       = int(sset[1]) - notZeroIndex
tag     = sset[3] 
CIT = [[c, s + idx, tag+ '-I']  for idx, c in enumerate(strAnno)]

CIT[-1][2] = tag + '-E'
CIT[0][2]  = tag + '-B'
    
if len(CIT) == 1:
    CIT[2] = tag + '-S'   
print(CIT)


CITAnnoText = []
for sset in SSETText:
    strAnno = sset[0]
    s       = int(sset[1]) - notZeroIndex
    tag     = sset[3] 
    CIT = [[c, s + idx, tag+ '-I']  for idx, c in enumerate(strAnno)]

    CIT[-1][2] = tag + '-E'
    CIT[0][2]  = tag + '-B'

    if len(CIT) == 1:
        CIT[0][2] = tag + '-S' 
        
    CITAnnoText.extend(CIT)
    
    
CITText = [[char, idx, 'O'] for idx, char in enumerate(strText)]

for citAnno in CITAnnoText:
    c, idx, t = citAnno
    assert CITText[idx][0] == c
    CITText[idx] = citAnno
    
CITText[:10]

`utils.getCITSents`

In [ ]:

strSents = ['结肠多发息肉。', '患中老年男性,慢性病程。']

CITText = [['结', 0, 'O'],
 ['肠', 1, 'O'],
 ['多', 2, '疾病-B'],
 ['发', 3, '疾病-I'],
 ['息', 4, '疾病-I'],
 ['肉', 5, '疾病-E'],
 ['。', 6, 'O'],
 ['\n', 7, 'O'],
 ['患', 8, 'O'],
 ['中', 9, 'O'],
 ['老', 10, 'O'],
 ['年', 11, 'O'],
 ['男', 12, 'O'],
 ['性', 13, 'O'],
 [',', 14, 'O'],
 ['慢', 15, '修饰-B'],
 ['性', 16, '修饰-E'],
 ['病', 17, 'O'],
 ['程', 18, 'O'],
 ['。', 19, 'O'],
 [' ', 20, 'O'],]



lenLastSent = 0
collapse    = 0
 
CITSents = []
for strSent in strSents:
    CITSent = []
    for sentTokenIdx, c in enumerate(strSent):
        # sentTokenIdx = txtTokenIdx - lenLastSent - collapse
        txtTokenIdx = sentTokenIdx + lenLastSent + collapse
        cT, _, tT = CITText[txtTokenIdx]
        while c != cT:
            collapse = collapse + 1
            txtTokenIdx = sentTokenIdx + lenLastSent + collapse
            cT, _, tT = CITText[txtTokenIdx]
            
        CITSent.append([c,sentTokenIdx, tT])
    lenLastSent = lenLastSent + len(strSent)
    CITSents.append(CITSent)
CITSents    

`utils.textLineReader` with `anno == 'embed'`

In [ ]:
from pprint import pprint

string = 'This is an annotated entity {{2018-12-22:date}}, try to extract it out!'

ST = [(block, 'O') if idx%2==0 else (block.split(':')[0], block.split(':')[-1]) 
    for idx, block in enumerate(string.replace("}}", '{{').split('{{'))]

pprint(ST)
# SSET, Str, S(char), E(char), Tag.

txtCharIdx = 0
SSET = []
strText = ''
for st in ST:
    string, tag = st
    sset = [string, txtCharIdx, txtCharIdx + len(string), tag]
    SSET.append(sset)
    txtCharIdx = sset[2]
    strText = strText + string
    
pprint(SSET)
pprint(strText) 

# Only Way to Check a SSET
for sset in SSET:
    assert sset[0] == strText[sset[1]: sset[2]]


# 7. Build Token String to Index Dict

In [ ]:
from nlptext.utils.infrastructure import UNK_ID, specialTokens, specialTokensDict, strQ2B, fileReader

##################################################################################################TOKEN_LTU
def buildTokens(tokenList, MaxTokenUnique = None):
    """
        Process raw inputs into a dataset.
        words: a list of the whole corpus
    """
    #########################################################################COUNT
    total_len_token = len(tokenList)
    print('The Total Number of Tokens:', total_len_token)
    print('Counting the number unique Tokens...          \t', datetime.now())
    if MaxTokenUnique:
        count = collections.Counter(tokenList).most_common(MaxTokenUnique)
    else:
        count = collections.Counter(tokenList).most_common()
    print('\t\tDone!')
    #########################################################################COUNT

    print('Generating Dictionary of Token Unique...\t', datetime.now())
    DTU = specialTokensDict.copy()
    DTU_freq = {sp_tk: 0 for sp_tk in specialTokens}
    for token, freq in count:
        if token is not specialTokens:
            DTU[token] = len(DTU)
            DTU_freq[token] = freq
        else:
            DTU_freq[token] = DTU_freq[token] + 1


    print('\t\tThe length of DTU is:', len(DTU), '\t', datetime.now())
    print('Generating the ORIGTokenIndex...       \t', datetime.now())
    data = np.zeros(len(tokenList), dtype = np.uint32)
    # data = []
    for idx, token in enumerate(tokenList):
        voc_id = DTU.get(token, UNK_ID)
        data[idx] = voc_id
        if voc_id == UNK_ID:
            DTU_freq[UNK] = DTU_freq[UNK] + 1

        # data.append(DTU.get(token,UNK_ID))
        if idx % 5000000 == 0:
            print('\t\tThe idx of token is:', idx, '\t', datetime.now())
    print('\t\tDone!')
    LTU = list(DTU.keys())

    if MaxTokenUnique:
        print('Only Keep First', MaxTokenUnique, 'Tokens.')
        print('The coverage rate is:', np.bincount(data)[UNK_ID]/total_len_token)
    # data = np.array(data)
    return data, LTU, DTU, DTU_freq
##################################################################################################TOKEN_LTU


# 8. From Corpus to Folders and From Folder to Texts


There are three methods

1. textFile

2. textLine

3. textBlock

In [ ]:
# STAGE 1
from pprint import pprint

from nlptext.utils.pyramid import CorpusFoldersReader, FolderTextsReaders

########### BOSON ###########
CORPUSPath = 'corpus/boson/'
corpusFileIden = '.txt'
textType   = 'line'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

assert anno == False or '.' in anno or anno == 'embed'
########################################################


MaxTextIdx = 10

Folders, CORPUSType = CorpusFoldersReader(CORPUSPath, iden = corpusFileIden)

pprint(Folders) # all possible files in this directory
pprint(CORPUSType)


for folderPath in Folders:
    print(folderPath)
    fileNames = Folders[folderPath]
    
    FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)
    
    for textIdx, strText_SSET_O_A in enumerate(FolderTexts):
        
        # we need to add some constraits to filter the textStrs
        strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
        print(textIdx, '--', strText)
        print(SSETText, '\n')
        if textIdx == MaxTextIdx:
            break
        
        # we need to add some constraits to filter the textStrs

# 9. Text to Sentences and Sentence to Tokens

In [ ]:
# STAGE 1
from pprint import pprint
from nlptext.utils.pyramid import CorpusFoldersReader, FolderTextsReaders

# STAGE 2
from nlptext.utils.pyramid import reCutText2Sent
from nlptext.utils.pyramid import segText2Sents, segSent2Tokens# (text, method = 'whole')
from nlptext.utils.pyramid import getCITSents
from nlptext.utils.pyramid import getCITText


########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed' # TODO
annoKW = {}


assert anno == False or '.' in anno or anno == 'embed'
########################################################
########################################################

MaxTextIdx = 10



########################################################
################   Things to Save   ####################
########################################################


CORPUS = {}
CORPUS['CORPUSPath'] = CORPUSPath
CORPUS['corpusFileIden'] = corpusFileIden # None if Dir else
CORPUS['CORPUSType']     = 'File' if corpusFileIden else 'Dir'
CORPUS['textType'] = textType

FOLDER = {}
FOLDER['folderPaths'] = [] 
FOLDER['NUMTexts'] = []
FOLDER['EndIDXTexts'] = []
        
TEXT = {}
TEXT['NUMSents'] = []
TEXT['EndIDXSents'] = []
TEXT['Text2SentMethod'] = Text2SentMethod
if textType == 'file':
    TEXT['ORIGFileName'] = []
if anno:
    TEXT['ANNOFileName'] = []
    
SENT = {}
SENT['NUMTokens'] = []
SENT['EndIDXTokens'] = []
SENT['Sent2TokenMethod'] = Sent2TokenMethod

TOKEN = {}
TOKEN['ORIGToken'] = []
TOKEN['TOKENLevel'] = TOKENLevel
if anno:
    TOKEN['ANNOToken'] = []

ANNO = {}
ANNO['anno'] = anno
ANNO['annoKW'] = annoKW

    
    
########################################################
##################     CHAINES      ####################
########################################################



###--> CHAIN: from Corpus to Folders <--###

CorpusFolders, CORPUSType = CorpusFoldersReader(CORPUSPath, iden = corpusFileIden)
assert CORPUS['CORPUSType'] == CORPUSType
pprint(CorpusFolders) # all possible files in this directory
pprint(CORPUSType)

for folderIdx, folderPath in enumerate(CorpusFolders):
    print(folderPath)
    fileNames = CorpusFolders[folderPath]
    
    ###--> CHAIN: from Folder to Texts <--###
    FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)
    
    for textIdx, strText_SSET_O_A in enumerate(FolderTexts):
        
        # we need to add some constraits to filter the textStrs
        strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
        
        print('\n', textIdx, '--', strText)
        print(SSETText, '\n')
        
        ###--> CHAIN: from strText to strSents <--###
        strSents = segText2Sents(strText, method = Text2SentMethod) # fixed
        
        for strSent in strSents:
            #- print(strSent)
            ###--> CHAIN: from strSent to strTokens <--###
            strTokens = segSent2Tokens(strSent, method = Sent2TokenMethod)
            
            ###--> CHAIN's End: Token itself <--###
            #- print(strTokens)
            TOKEN['ORIGToken'].extend(strTokens)
            
            lenSent = len(strTokens)
            SENT['NUMTokens'].append(lenSent)
            try:
                SENT['EndIDXTokens'].append(SENT['EndIDXTokens'][-1] + lenSent)
            except:
                SENT['EndIDXTokens'].append(lenSent)
            
        
        lenText = len(strSents)
        TEXT['NUMSents'].append(lenText)
        try:
            TEXT['EndIDXSents'].append(TEXT['EndIDXSents'][-1] + lenText)
        except:
            TEXT['EndIDXSents'].append(lenText)
            
        if origTextName:
            TEXT['ORIGFileName'].append(origTextName)
            
            
        ########################################
        if anno:
            # TOKEN['ANNOToken'] = []
            # assert SSETText   != [] # May occur Errors
            for sset in SSETText:
                assert sset[0] == strText[sset[1]: sset[2]]
            if SSETText == []:
                print('\nThe SSET of this Text is Empty!!!')
                print(strText, '\n') # to check what happen
                    
            ############### PART One: Get CITText ###########
            #
            # CITText  = foo1(strText, SSETText)
            # 
            CITText = getCITText(strText, SSETText)

            
            
                
            ############### PART TWO: Get CITSents ###########
            #
            # CITSents = foo2(strSents, CITText)
            #
            CITSents = getCITSents(strSents, CITText)
           

            ############### PART THREE: Get TOKEN['ANNOToken'] ###########
            #
            # TOKEN['ANNOToken'] = foo3(CITSents, strSents)
            #
            for sentIdx, CITSent in enumerate(CITSents):
                
                # Corporate into TOKEN['ANNOToken']
                # pay attention here, CIT is char-based, but TOKEN may be word-based.
                # strTokens = segSent2Tokens(strSent, method=Sent2TokenMethod)
            
                if TOKENLevel == 'char':
                    TOKEN['ANNOToken'].extend([CITToken[2] for CITToken in CITSent])
                    #- pprint(sentIdx)
                    #- pprint(CITSent)
                else:
                    # TODO
                    pass 
            # save the file
            
            if annoTextName:
                TEXT['ANNOFileName'].append(origTextName)
            
        
        if textIdx == MaxTextIdx:
            break
    
    # Back to Folder
    lenFolder = textIdx
    FOLDER['folderPaths'].append(folderPath)
    
    FOLDER['NUMTexts'].append(lenFolder) # to remove
    try:
        FOLDER['EndIDXTexts'].append(FOLDER['EndIDXTexts'][-1] + lenFolder)
    except:
        FOLDER['EndIDXTexts'].append(lenFolder)
        
# End here
lenCorpus = folderIdx
CORPUS['NUMFolders'] = [lenCorpus]
CORPUS['EndIDXFolders'] = [lenCorpus]

# 10. Test

In [ ]:
from pprint import pprint
from nlptext.base import BasicObject

########### BOSON ###########
CORPUSPath = 'corpus/boson/'
corpusFileIden = '.txt'
textType   = 'line'
Text2SentMethod = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}


MaxTextIdx = False


BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW, MaxTextIdx)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

# DictToken = corpus.DictToken
# print(DictToken)

In [ ]:
print(corpus.CORPUS)

In [ ]:
print(corpus.FOLDER)

In [ ]:
print(corpus.TEXT)

In [ ]:
print(corpus.SENT)

In [ ]:
print(corpus.TOKEN)

In [ ]:

import numpy as np
txtIdxes = list(set(list(np.random.randint(corpus.TEXT['length'], size = 10))))
txtIdxes

In [ ]:
# print(corpus.Folders)
# print(corpus.FOLDER)
# print(corpus.Texts)
# print(corpus.TEXT)
# print(corpus.Sentences)
from nlptext.text import Text

sentIdx = 0
for txtIdx in txtIdxes:
    
    txt = Text(txtIdx)
    print('\n', txt, '\n')
    for st in txt.Sentences:
        print(sentIdx, '-->',st.sentence)
        sentIdx = sentIdx + 1

In [ ]:
st = corpus.Sentences[31]
st.Tokens

In [ ]:
BasicObject.TokenNum_Dir

In [ ]:
from nlptext.text import Text
loc_idx = 9  # how to interpret loc_idx: support there are n texts in the whole corpus, get the loc_idx th text
txt = Text(loc_idx)
txt.Tokens